# Overview

During this session, you will participate in a supervised learning exercise about digit recognition. You will:
- Build a neural network.
- Train your neural network with a dataset that contains images that represent numbers. 
- Modify the architecture of the neural network to obtain better results.

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

# The MNIST data


The MNIST data is comprised of pictures that represent a number and includes the number label associated to each picture. The data set is split into three data parts:
- training (mnist.train)
- testing (mnist.test)
- validation (mnist.validation)

The validation split is important because it's essential in machine learning that there is separate data which is not given to the machine during the learning phase. After the initial machine learning session, you present the separate data to the machine and assess the performance of it. The test and validation sets may serve different evaluation purposes.

In [2]:
%%capture
mnist = input_data.read_data_sets("../Data/MNIST_data/", one_hot=True)

# Creating the model

Common numerical computing libraries in Python use external code that is implemented in other languages to take advantage of their efficiency. However, switching back to Python for every operation causes overhead. This overhead is bad if you want to run computations on GPUs or in a distributed manner since there is a high cost to transfer data.

TensorFlow does its heavy lifting outside of Python. Although it does not run expensive operations independently from Python, TensorFlow enables you to describe a graph of interacting operations that run entirely outside Python.

#### TensorFlow Graph
Create a TensorFlow graph that represents a neural network with no hidden layers and an output layer comprised of 10 nodes.

In [5]:
# First Attempt - Slides
image_size = 28*28
number_digits = 10
next_layer_neurons = 10

x = tf.placeholder(tf.float32, [None, image_size])
y_ = tf.placeholder(tf.float32, [None, number_digits])
W = tf.Variable(tf.zeros([image_size, next_layer_neurons]))
b = tf.Variable(tf.zeros([next_layer_neurons]))
y = tf.matmul(x, W) + b

In [6]:
# Second Attempt - Change Initialization
image_size = 28*28
number_digits = 10
next_layer_neurons = 10

x = tf.placeholder(tf.float32, [None, image_size])
y_ = tf.placeholder(tf.float32, [None, number_digits])
W = tf.Variable(tf.truncated_normal(shape=[image_size, next_layer_neurons],
                                    mean=0.0,
                                    stddev=1.0))
b = tf.Variable(tf.truncated_normal(shape=[next_layer_neurons],
                                    mean=0.0,
                                    stddev=1.0))
y = tf.matmul(x, W) + b

In [7]:
# Third Attempt - Add a Hidden Layer
image_size = 28*28
number_digits = 10
next_layer_neurons = 300

x = tf.placeholder(tf.float32, [None, image_size])
y_ = tf.placeholder(tf.float32, [None, number_digits])

# Hidden Layer
shape1 = [image_size, next_layer_neurons]
W1 = tf.Variable(tf.truncated_normal(shape=shape1,
                                     mean=0.0,
                                     stddev=1.0))
b1 = tf.Variable(tf.truncated_normal(shape=[next_layer_neurons],
                                     mean=0.0,
                                     stddev=1.0))
y1 = tf.matmul(x, W1) + b1

# Output Layer
shape2 = [next_layer_neurons, number_digits]
W2 = tf.Variable(tf.truncated_normal(shape=shape2,
                                     mean=0.0,
                                     stddev=1.0))
b2 = tf.Variable(tf.truncated_normal(shape=[number_digits],
                                     mean=0.0,
                                     stddev=1.0))
y = tf.matmul(y1, W2) + b2

In [8]:
# Fourth Attempt - Add Sigmoid
image_size = 28*28
number_digits = 10
next_layer_neurons = 500

x = tf.placeholder(tf.float32, [None, image_size])
y_ = tf.placeholder(tf.float32, [None, number_digits])

# Hidden Layer
shape1 = [image_size, next_layer_neurons]
W1 = tf.Variable(tf.truncated_normal(shape=shape1,
                                     mean=0.0,
                                     stddev=1.0))
b1 = tf.Variable(tf.truncated_normal(shape=[next_layer_neurons],
                                     mean=0.0,
                                     stddev=1.0))
Y1 = tf.sigmoid(tf.matmul(x, W1) + b1)

# Output Layer
shape2 = [next_layer_neurons, number_digits]
W2 = tf.Variable(tf.truncated_normal(shape=shape2,
                                     mean=0.0,
                                     stddev=1.0))
b2 = tf.Variable(tf.truncated_normal(shape=[number_digits],
                                     mean=0.0,
                                     stddev=1.0))
y = tf.matmul(Y1, W2) + b2

# Defining Cost Funtion and Optimizer

In order to train the model, define what it means to improve the results after each iteration. Use a cost function and try to minimize with respect to it. The cost function represents how far you are from our desired outcome. Minimizing the error leads you towards improving the model.

A common cost function is called *cross-entropy*. Cross-entropy takes advantage of large errors and reduces the learning slow down that is caused by using traditional cost functions (i.e. quadratic cost function). In summary, it will take less to train a good model.

#### TensorFlow
1. Create a tensor to represent the cross-entropy function. 
1. Create a tensor to represent a Gradient Descent Optimizer that minimizes the cross-entropy.

In [9]:
# First Attempt - Slides
cross_entropy = tf.reduce_mean(
  tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

learn_rate = 0.5
train_step = tf.train.GradientDescentOptimizer(learning_rate=learn_rate) \
             .minimize(cross_entropy)

In [10]:
# Second Attempt - Change Optimizer
cross_entropy = tf.reduce_mean(
  tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

learn_rate = 0.01
train_step = tf.train.AdamOptimizer(learning_rate=learn_rate).minimize(cross_entropy)

# Creating a TensorFlow Session

You have defined your model by creating a complete Tensorflow graph. Now, you need to launch it. Create an interactive session and initialize all the variables defined before.

#### Interactive Session
Create an Interactive Session and run the global variable initializer.

In [11]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Train

MNIST is a large dataset. To train using a batch learning method is too time intensive in-between epochs. Therefore, use small batches of random data. This method is called stochastic training.

#### Stochastic Training
1. In a `for` loop, take 100 random samples from MNIST and run the train step using the resulting batches. 
1. Repeat the process as many times as necessary. 
1. Present all the training datasets.


In [12]:
# First Attempt - Slide
# An epoch consists of running through all 55,000 observations once
iterations = 55
batch_size = 1000
for _ in range(iterations):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [13]:
# Second Attempt - Juggle Values
# An epoch consists of running through all 55,000 observations once
epochs = 10
iterations = 1000
batch_size = 55
for epoch in range(epochs):
    for _ in range(iterations):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

# Evaluate your model

To understand your model's precision, you need to compare your results with the expected output. To calculate the precision, you need to sum the correct classifications over the size of the testing dataset.
#### Calculating Precision
1. Create a Tensor that compares the model's output with the expected output. 
1. Determine the fraction that are correct.

In [8]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels}))

0.97


# Summary

During the exercise you:
- Learned how to create a neural network and train it. 
- Tested the performance of your model, and found that the network architecture definition is important to obtain better results.
- Learned the importance of the initialization step (random numbers instead of zeroes), and how it impacts performance. 
- Reviewed the activation functions’ impact in performance.